<a href="https://colab.research.google.com/github/tobiasgobel/VQE_project/blob/master/VQE_grid_opt_minimize_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages

In [1]:
from sympy.series import series
from scipy.optimize import minimize, NonlinearConstraint
import time
import itertools
from sympy import symbols, Matrix, SparseMatrix, cos, sin, expand, lambdify, O
from sympy.utilities.iterables import multiset_permutations
from scipy.linalg import expm, sinm, cosm
from functools import *
from operator import *
import scipy
import sympy
import numpy as np
import random
import math
from numba import jit
import matplotlib.pyplot as plt

In [2]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' %(f.__name__, te-ts))
        return result
    return wrap

# Defining functions

In [3]:

def pauli_on_pauli(p1,p2):
    
    if p1 == 'X' and p2 == 'Y':
        return 1j, 'Z'
    elif p1 == 'X' and p2 == 'X':
        return 1, 'I'
    elif p1 == 'Y' and p2 == 'Y':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'Z':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'X':
        return 1j, 'Y'
    elif p1 == 'Z' and p2 == 'Y':
        return -1j, 'X'
    elif p1 == 'I':
        return 1, p2
    elif p2 == 'I':
        return 1, p1
    else:
        a, p = pauli_on_pauli(p2,p1)
        return -1*a, p

def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return((spin+1)%2, 1)
    elif pauli=='Y':
        return((spin+1)%2, 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)

def findCombinationsUtil(li, arr, index, num, reducedNum):
    z = []
    if (reducedNum < 0): 
        return; 
    if (reducedNum == 0): 
  
        for i in range(index): 
            z = z + [arr[i]]
        li.append(z) 
        return;

    prev = 1 if (index == 0) else arr[index - 1]; 
  
    for k in range(prev, num + 1): 
          

        arr[index] = k; 
  
        findCombinationsUtil(li,arr, index + 1, num,  
                                 reducedNum - k); 
    return li

def k_all(N, generators, order): 
      
    # array to store the combinations 
    # It can contain max n elements
    out = []
    k_length = len(generators)
    for k in range(1, order+1):
        arr = [0] * k;
        output = []
        a =  findCombinationsUtil([], arr, 0, k, k);
        for i in a:
            if len(i)<= k_length:
                i = i.extend((k_length-len(i))*[0])
        for j in a:
            if len(j) == k_length:
#                 if k_vector(N, interactions,j).state()[1] != N*[0]:
                output = output + list(multiset_permutations(j))
        out =  out+ output
    return [tuple(p) for p in [[0]*k_length] + out]

def power_product(x,y):
    out = 1
    for i in range(len(x)):
         out*= x[i]**y[i]
    return out



In [4]:
@timing
@jit(nopython=True)
def Energy_eigen(H):
  result = np.linalg.eig(H)
  index = np.argmin(result[0])
  return result[0][index],result[1][index]

def Energy_matrix(thetas,N,H,ansatz, K):

  #build psi
  a = np.eye(2**N)
  zero_state = np.zeros(2**N)
  zero_state[0]=1
  for i in range(len(ansatz)-1,-1,-1):
    T = ansatz[i]
    exp = expm(1j*(np.pi/4*K[i]+thetas[i])*T)
    a = a @ exp
  

  psi = a @ zero_state
  
  #build Hamiltonian
  Energy = (np.transpose((np.conj(psi)) @ (H @ (psi))))

  return np.real(Energy)

def psi(thetas, ansatz,K):
    #build psi
  a = np.eye(2**N)
  zero_state = np.zeros(2**N)
  zero_state[0]=1
  for i in range(len(ansatz)-1,-1,-1):
    T = ansatz[i]
    exp = expm(1j*(np.pi/4*K[i]+thetas[i])*T)
    a = a @ exp
  

  psi = a @ zero_state
  
  return psi

  

In [5]:
Pauli = {"X":np.array([[0,1],[1,0]]), "Z": np.array([[1,0],[0,-1]]),"Y":np.array([[0,-1j],[1j,0]]), "I":np.eye(2)}

class pauli:
  def __init__(self,string, N, factor = 1):
    self.string = string
    self.factor = factor
    self.N = N
    self.starting_state = np.array([0]*self.N)


  def __str__(self):
    return self.string+".   factor: "+str(self.factor)
    
  #define multiplying by a constant (on left hand side)
  def __rmul__(self, c):
    return pauli(self.string,self.N, c*self.factor)

  #define the power of a pauli string
  def __pow__(self, c): 
    C = pauli("I0",self.N)
    for i in range(abs(c)):
      C = C*self
    return C

  #define multiplying two pauli strings
  def __mul__(self, x):
    pos1, pauli1 = self.split()
    pos2, pauli2 = x.split()
    factor = self.factor*x.factor
    string = ""
    counter1 =0
    counter2 =0

    for j in range(self.N):
      end1 = counter1 == len(pos1)
      end2 = counter2 == len(pos2)

      if not end1 and not end2:
        if int(pos1[counter1]) == j and int(pos2[counter2]) == j:
          a, p= pauli_on_pauli(pauli1[counter1],pauli2[counter2])
          factor *= a
          string+= p+str(j)
          counter1+=1
          counter2+=1
        elif int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
        elif int(pos2[counter2]) == j:
          string+=pauli2[counter2]+str(j)
          counter2+=1
      elif not end1:
        if int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
      elif not end2:
          if int(pos2[counter2]) == j:
            string+=pauli2[counter2]+str(j)
            counter2+=1
      else:
        pass
      
    return pauli(string, self.N, factor)

  #calculate resulting state of paulistring when acted upon initial_state  
  def state(self, initial_state = 0):
    pos, pauli = self.split()
    init_state = self.starting_state + initial_state
    a = self.factor
    for j in range(len(pos)):
      Pauli = pauli[j]
      spin = init_state[int(pos[j])]
      new_spin, factor = single_pauli_action(Pauli,spin)
      init_state[int(pos[j])] = new_spin
      a *= factor
    return a, tuple(init_state)

    
#creating lists of operators and corresponding positions
  def split(self):
    pauli_lst = []
    pos_lst = []
    prev_int = False
    for k in self.string:
        if k.isdigit():
            if not prev_int:
                pos_lst.append(k)
            else:
                pos_lst[-1] += k
            prev_int = True
        else:
            pauli_lst.append(k)
            prev_int = False
    return pos_lst, pauli_lst
  
  def matrix_repr(self):
    positions, paulis = self.split()
    Kron = 1
    counter = 0
    for j in range(self.N):
      if counter == len(positions):
        for _ in range(self.N-j):
          Kron = np.kron(Kron, np.eye(2))
        break

      elif j == int(positions[counter]):
        Kron = np.kron(Kron, Pauli[paulis[counter]])
        counter+=1
      else:
          Kron = np.kron(Kron, np.eye(2))

    return Kron*self.factor
  
print(pauli("X0X1Y2",3,7).matrix_repr())
  

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-7.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+7.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.-7.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+7.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.-7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+7.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [6]:
#gives result of transformation exp(-i*T1)*T2*exp(i*T2)
def Clifford_map(T1, T2, reversed_arguments = True):
  global ansatz
  T1T2 = T1*T2
  T2T1 = T2*T1
  if T1T2.factor == T2T1.factor:
    if reversed_arguments:
      return T1
    else:
      return T2
  elif T1T2.factor == -T2T1.factor:
    if reversed_arguments:
      return -1j*T2T1
    else:
      return -1j*T1T2
  else:
    return "something wrong here"


#returns list of pauli objects that are the result 
#of pulling all clifford gates to the left
def pull_cliffords_through(ansatz, K, N):
  T_K = [ansatz[0]]
  
  for j in range(1, len(ansatz)):
    T = ansatz[j]
    for i in range(j-1,-1,-1):
      for _ in range(abs(K[i])):
        T = Clifford_map(T,np.sign(K[i])*ansatz[i])
    T_K += [T] 
  return T_K





In [7]:

def s_dict(N, ansatz, K, order):
  start = time()
  s_dict = {} #keys: possible bitstrings, values dictionary with orders
  T_K = pull_cliffords_through(ansatz, K, N)
  K_all = list(map(tuple, itertools.product([0, 1], repeat=len(ansatz))))

  for i in K_all: #loop through all
    if sum(i) > order:
      continue
    
    #calculate state that is produced by T_i
    pauli_string = power_product(T_K[::-1], i[::-1])
    factor, state = pauli_string.state()
    #calculate magnitude of term
    term = (1j)**sum(i)*factor
    #check whether binary string is in dictionary, otherwise add
    if state not in s_dict:
      s_dict[state] = ([list(i)],[term])
    else:
      current = s_dict[state]
      current[0].append(list(i))
      current[1].append(term)

  #make np.array
  for st in s_dict:
    lst = s_dict[st]
    s_dict[st] = (np.array(lst[0]),np.array(lst[1]))
  
  return s_dict


def G_k(N, H, ansatz, K):
  g_k = []

  #Initialize list of Clifford gates with respective power of K.
  G_K = []
  for i in range(len(K)):
    G_K += [np.sign(K[i])*ansatz[i]]*abs(K[i])
  for P in H:
    # G_K = [ansatz[i]**K[i] for i in range(len(K))]
    #Apply nested Clifford Map to obtain G^-K P_a G^K
    paulistring = reduce(Clifford_map, [P]+G_K[::-1])
    g_k += [paulistring]
  return g_k

@jit(nopython=True)
def dict_multiplication(k,values,thetas):
  sum = 0
  for i in range(k.shape[0]):
    product = 1
    for j in range(k.shape[1]):
      product*=(np.cos(thetas[j]))**(-k[i,j]+1)*(np.sin(thetas[j]))**k[i,j]
    sum += product*values[i]
  return sum

def Normalize(s_d, thetas, order):
  sum = 0
  for s in s_d:
    k, values = s_d[s]
    k1, values1 = s_d[s]
    factor = dict_multiplication(k,values,thetas)
    factor1 = dict_multiplication(k1,values1,thetas)
    sum += np.conj(factor1)*factor

  return sum



In [8]:
 
def energy(thetas,ansatz, s_dict,G_K, order):
  E = 0
  s_dict1 = s_dict
  for paulistring in G_K: #loop through terms in Hamiltonian
    E_a = 0
    #loop over basis states
    for s in s_dict1:
      E_a_s = 0
    
      #Calculate G^-K P_a G^K |s>
      a, state = paulistring.state(s)
      #Define contributions of |s> and |s'>
      psi_s1 = s_dict1[s]

      #Check if the state created by hamiltonian, exists in wavefunction
      try:
        psi_s2 = s_dict1[state]
      except:
        break

      A = dict_multiplication(psi_s1[0],psi_s1[1],thetas)
      B = dict_multiplication(psi_s2[0],psi_s2[1],thetas)
      E_a_s = A*np.conj(B)

      E_a_s *= a
      E_a += E_a_s
    E += E_a
  
  norm = Normalize(s_dict1, thetas, order)
  # print(f"E:{E}, Norm{norm}, E_final:{E/norm}")
  return np.real(E/norm)

In [9]:
def angle_compare(theta_opt, theta_appr, K):
  theta_appr = theta_appr + np.array(K)*np.pi/4
  theta_appr = theta_appr % (2*np.pi)
  theta_opt = theta_opt % (2*np.pi)

  distance = np.linalg.norm(theta_opt-theta_appr)
  
  return distance

def wavefunction_compare(theta_opt, theta_appr, K, ansatz):
  wave_1 = psi(theta_opt, ansatz, [0]*len(ansatz))
  wave_2 = psi(theta_appr, ansatz, K)

  print(f"wavefunction exact: {wave_2}, wavefunction_approx: {wave_1}")
  overlap = np.abs(np.conj(wave_1) @ wave_2)
  return overlap

def local_to_global_angle(thetas, K):
    thetas = thetas + np.array(K)*np.pi/4
    thetas = thetas % (2*np.pi)
    return thetas
    
def global_to_local_angle(thetas, K):
    thetas = thetas - np.array(K)*np.pi/4
    thetas = thetas % (2*np.pi)
    return thetas


# Pseudo-code for K-cell finding

K-cell finding -- old version
```
set number of iterations: iterations = iters
set initial K-cell: K_init = [0]*len(ansatz)
set order
set epsilon

for i in range(iters):

  calculate s_dict
  calculate G_K

  result = scipy.minimize(Energy, order, args = (s_dict, G_K)

  if biggest_angle > 1/8* pi:
      update K_init at index of biggest angle
  elif biggest_angle - 1/8* pi < epsilon:
      update K_init at index of biggest_angle
  else:  (all_angles < 1/8*pi)
    --> Valid K_cell
    return result

  if K_init in K_path:
      if there exists next_biggest angle > 1/8 * pi:
          update K_init at next_biggest angle
      elif previous K_init has next biggest angle > 1/8*pi:
          K_init = prev_K_init updated at next_biggest angle
      elif periodicity == 2:
          take average angle between two K-cells
      elif periodicity > 2:
          Output angles with lowest |theta|

(if run for all iterations without finding K-cell, take K_cell with lowest |theta|)
```







K-Cell finding ---- Minimizing Energy (before Number of magic gates)

```
set number of K-cells one wants to consider: N_K
set K #initial K-cell
set order # the order of energy approximation

K_path = []#list of all visited K-vectors (may have different nodes)

K_tree = {(0,0):{"K":K, "seen":False}}

N_magic_prev = len(angles) #max magic gates
Energy_prev = inf #initialize with infinity

E_epsilon = 1e-5 #how close should Energy be to continue exploring branch
curr_node = (0,0) #initialize first node


#loop over number of iterations
for _ in range(N_K):
    

    calculate s_dict
    calculate G_K

    #minimize for current K-vector
    result = minimize(energy, order, args = (s_dict, G_K)) #bounded by |pi/8|


    N_magic = number of angles epsilon close to pi/8
    Energy = result.fun

    #add data to K_tree dictionary
    K_tree[curr_node]["seen"] = True
    K_tree[curr_node]["N_magic"]=N_magic
    K_tree[curr_node]["Energy"]=Energy

    #add K to calculated K-vectors
    K_path += [K.copy()]

    

    # if energy increases, kill branch
    # if energy only increases by small margin, keep going

    if Energy - Energy_prev > E_epsilon:
        curr_node, K = find_new_branch()
        continue


    #include new branches 
    for i in range(N_magic):
        K_new_i = K but +/- 1 at i^th magic angle
        K_tree[curr_node+(i,)]={"K":K_new_i, "seen":False}

    #all angles within K-cell
    if N_magic == 0:
        break

    #choose new branch, only runs if energy decreased

    updated = False
    for i in range(N_magic):
        if K_i not in K_path:
            curr_node += (i,)
            K = K_tree[curr_node]["K"]
            updated = True
            break
    if not updated: #Only repeated gates, then move up again
        curr_node, K = find_new_branch()

        
def find_new_branch(K_tree, curr_node, K_path):
    node_up = curr_node[:-1]

    if node_up == (0,0):
        #whole tree has been explored, return K with lowest energy
        return K | argmin(E(K))


    N_magic = K_tree[node_up]["N_magic"]

    updated = False

    for i in range(N_magic):
        node_i = node_up+=(i,)
        if node and K haven't been seen:
            return node_i, K_tree[node_i]["K"]
    if not updated:
        find_new_branch(K_tree, node_up, K_path)

```


#Code for K-cell finding

In [10]:
def find_new_branch(K_tree, node, K_path, shuffle = False):

    node_up = node[:-1]
    if node == (0,):
        return "whole tree has been searched"
    N_magic = K_tree[node]["N_magic"]
    
    updated = False

    #randomly shuffle magic gates
    magic_gates = list(range(N_magic))
    if shuffle:
        random.shuffle(magic_gates)

    #loop through randomly ordered magic-gates
    for i in magic_gates:
        tree_i = node+(i,)
        K_i = K_tree[tree_i]["K"]
        if not K_tree[tree_i]['seen'] and K_i not in K_path:
            updated = True
            return tree_i

    if not updated:
        return find_new_branch(K_tree, node_up, K_path)

def output(K_tree, optim_node, termination, global_min, ansatz, N, H, log = True):
    #get node with lowest energy
    K_best = K_tree[optim_node]
    angles = K_best['angles']
    K = K_best['K']

    #print information about best K-cell
    if log:
        print("\n")
        print("-"*40)
        print("\n RESULT \n")
        print(f"{'Termination cause' + ':':<25}{f'{termination}'}\n")
        for key, value in K_best.items():
            print(f"{f'{key}:':<25}{f'{value}'}\n")
        print('\nCompare result to global minimum\n')
        print(f"{'Minimum E_approx of ansatz:':<25} {f'{global_min.fun}'}\n")
        print(f"{'Angles distance:':<25} {f'{angle_compare(global_min.x,angles,K)}'}\n")
        print(f"{'Wavefunction overlap:':<25} {f'{wavefunction_compare(global_min.x, angles, K, ansatz)}'}\n")
        print(f"Minimum E ansatz: {Energy_matrix(angles,N,H,ansatz,K)}")
        


    return K_best
    



In [44]:
#initialize

# N = 2
# Z_h = -1
# X_h = -.1
# ansatz = [pauli("X0X1",N),pauli("Z0",N),pauli("Z1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h)]
# H+= [pauli("X0X1",N,X_h)]

# N = 3
# Z_h = -1
# X_h = -.1
# ansatz = [pauli("X0X1",N), pauli("X1X2",N),pauli("Z0",N),pauli("Z1",N),pauli("Z2",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h)]


# N = 3
# Z_h = -1
# X_h = -10
# ansatz = [pauli("Y0X1",N),pauli("Y1X2",N)]*2
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h)]

N = 4
Z_h = -1
X_h = -10
ansatz = [pauli("Y0X1",N),pauli("Y1X2",N),pauli("Y2X3",N)]*3
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h)]
H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h)]


H_m = sum([h.matrix_repr() for h in H])
ansatz_m = [a.matrix_repr() for a in ansatz]



N_K = 50
K = list(np.random.randint(4, size = len(ansatz)))
theta_init = [1]*len(ansatz)
order = len(ansatz)*2
K_tree = {(0,):{"K":K,"angles":theta_init},(0,0):{"K":K, "seen":False}}
N_magic_prev = len(ansatz) #max magic gates
Energy_prev = np.inf
termination = "Maximum number of iters has been reached"

curr_node = (0,0)
optim_node = (0,0) #node with lowest energy

# boundary = "hypercube"
boundary = "hypersphere"

if boundary == "hypercube":#set boundary to hypercube
    bounds = [(-np.pi/8,np.pi/8)]*len(ansatz)

elif boundary == "hypersphere":#set boundary to hypersphere

    #define constraint of hypersquare
    def constraint(thetas):
        corner = np.sqrt(len(thetas)*(np.pi/8)**2)
        norm = np.linalg.norm(thetas)
        return corner - norm

    #define constraint to pass into scipy.minimize
    con = {'type':"ineq", 'fun':constraint}
        

epsilon = 1e-3
E_epsilon = 1e-5
K_path = []
log = True

#calculate global minimum
global_min = scipy.optimize.minimize(Energy_matrix, theta_init, jac = False, args = (N,H_m,ansatz_m, K))

if log: print("LOG")

for iter in range(N_K):

    K = K_tree[curr_node]["K"]

    s = s_dict(N, ansatz, K, order)
    G_K = G_k(N, H, ansatz,K)



    #previous minimized set of angles
    node_above= K_tree[curr_node[:-1]]
    prev_angles_global = local_to_global_angle(node_above["angles"], node_above["K"])

    #translate to angles in current K_cell
    init_angles = global_to_local_angle(prev_angles_global, K)

    #calculate energy
    if boundary == "hypercube": #boundary set to cube
        result = scipy.optimize.minimize(energy, init_angles,jac = False, args = (ansatz,s,G_K,order),bounds=bounds)

    elif boundary == "hypersphere": #boundary set to sphere
        result = scipy.optimize.minimize(energy, init_angles,jac = False, args = (ansatz,s,G_K,order),constraints=con)
    else:
        raise ValueError("Invalid boundary condition was given. hypercube or hypersphere")

    #get indices of magic gates
    magic_indices = np.where(np.pi/8 -  np.abs(result.x)< epsilon)

    N_magic = len(magic_indices[0])
    Energy = result.fun


    #add data to dictionary
    K_tree[curr_node]["seen"] = True
    K_tree[curr_node]["N_magic"]=N_magic
    K_tree[curr_node]["energy"] = Energy
    K_tree[curr_node]["angles"] = result.x
    K_tree[curr_node]["angles_norm"] = np.linalg.norm(result.x)

    #add K-vector corresponding to curr_node to K_path
    K_path += [K.copy()]

    if log:
        print("-"*30)
        print(f"iteration: {iter}")
        for key, value in K_tree[curr_node].items():
            print(f"{f'{key}:':<25}{f'{value}'}")

    #if Energy increases, find new branch skip iteration
    if Energy - Energy_prev > E_epsilon:
        new_node = find_new_branch(K_tree, curr_node[:-1], K_path)
        if type(new_node)!=tuple:
            termination = "Whole tree has been explored"
            break
        else: 
            curr_node = new_node
            continue


    #include new branches
    for i in range(N_magic):
        magic_index = magic_indices[0][i]
        sign = np.sign(result.x[magic_index])
        K_i = K_tree[curr_node]["K"].copy()
        K_i[magic_index] = K_i[magic_index]+int(sign)*1
        K_tree[curr_node+(i,)]={"K":K_i, "seen":False}
    
    #choose new branch, only fires when Energy decreases or slightly increases
    assert Energy - Energy_prev < E_epsilon, "Energy > Energy_prev??"

    #update best node, if energy decreased
    if Energy - Energy_prev < 0:
        optim_node = curr_node

    #update current node
    new_node = find_new_branch(K_tree, curr_node, K_path)
    if type(new_node)!=tuple:
        termination = "Whole tree has been explored"
        break
    else:
        curr_node = new_node

    #udpate previous magic gates
    N_magic_prev = N_magic
    
    #update previous energy
    Energy_prev = Energy

#ouput
out = output(K_tree, optim_node, termination, global_min,ansatz_m,N,H_m, log)






LOG
------------------------------
iteration: 0
K:                       [1, 3, 0, 0, 2, 3, 0, 3, 1]
seen:                    True
N_magic:                 4
energy:                  -28.848003030168798
angles:                  [-0.0049582   0.04612452  0.58102238 -0.41188182  0.59308591 -0.03594628
  0.61525381 -0.01443321  0.38306511]
angles_norm:             1.1780972694794298
------------------------------
iteration: 1
K:                       [1, 3, 1, 0, 2, 3, 0, 3, 1]
seen:                    True
N_magic:                 2
energy:                  -29.916222341945115
angles:                  [-0.0033159  -0.05210357 -0.05026944  0.37441797 -0.74140575  0.01663191
 -0.74432726  0.05420277 -0.36817395]
angles_norm:             1.178097528166931
------------------------------
iteration: 2
K:                       [1, 3, 1, 0, 1, 3, 0, 3, 1]
seen:                    True
N_magic:                 4
energy:                  -21.98567558284
angles:                  [ 0.02211145  0.061

In [41]:
theta = np.random.random(len(ansatz))*.2
K = np.random.randint(3, size = len(ansatz))-1
print(K)
E = Energy_matrix(theta, N, H_m, ansatz_m, K)

order = len(ansatz)*2
s = s_dict(N, ansatz, K, order)
G_K = G_k(N, H, ansatz,K)

E_approx = energy(theta, ansatz, s, G_K, order)
print(E - E_approx)

[ 1 -1 -1  1  1 -1  1  0  1]
7.105427357601002e-15


#Questions



*   If cells repeat, choose one, end algorithm
*   What do if theta --> pi/8?


*   How many gates are realistically possible on physical quantum circuit? How many K-cells?
* Where does the randomness come from?
* Why does the energy get lower than the actual ground state energy?



* Compute global minimum of ansatz
* Compute corresponding angles
* Compare angles global minimum to approximated ( multiple minima ?)--> look at psi instead of the angles?

* switch to next biggest angle if periodic behaviour appears

* Return value when algorithm finds a minimum within a K-cell, that has up until that point the lowest energy? Or keep on exploring?

* 





# Tests

In [ ]:

N = 10
ansatz = [pauli("X0Y1",N),pauli("X1Y2",N),pauli("X2Y3",N),pauli("X3Y4",N),pauli("X4Y5",N),pauli("X5Y6",N),pauli("X6Y7",N),pauli("X7Y8",N),pauli("X8Y9",N)]*2
Z_h = -1
X_h = -1
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("Z4",N,Z_h),pauli("Z5",N,Z_h),pauli("Z6",N,Z_h),pauli("Z7",N,Z_h),pauli("Z8",N,Z_h),pauli("Z9",N,Z_h)]
H+=[pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h),pauli("X4X5",N),pauli("X5X6",N),pauli("X6X7",N),pauli("X7X8",N),pauli("X8X9",N)]
K = [0]*18


# N = 7
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0Y1",N),pauli("Z1Z2",N), pauli("X2Y3",N),pauli("X3Y4",N),pauli("X4Y5",N),pauli("Z5Y6",N)]*2
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("X2",N,Z_h),pauli("Y3",N,Z_h),pauli("Z4",N,Z_h),pauli("Z5",N,Z_h),pauli("Y6",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h),pauli("X3X4",N,X_h),pauli("X4X5",N,X_h),pauli("X5X6",N,X_h)]
# K = [0]*len(ansatz)


# N = 3
# Z_h = -1
# X_h = -.01
# ansatz = [pauli("Z0Y1",N),pauli("X1Y2",N),pauli("Z0Z1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("X2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N,X_h)]
# K = [0,1,1]





# N = 2
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0Y1",N),pauli("Z0I1",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h)]
# H= [pauli("X0X1",N,X_h)]
# K = [0,1]



# N = 1
# Z_h = -1
# X_h = -1
# ansatz = [pauli("X0",N),pauli("Y0",N)]
# H = [pauli("Y0",N,Z_h)]
# K = [1,0]


matrix_ansatz = [t.matrix_repr() for t in ansatz]
matrix_H = sum([h.matrix_repr() for h in H])



In [ ]:

thetas = np.random.random(len(ansatz))*np.pi/4
# thetas = [np.pi/4,np.pi/4]
# thetas = np.ones(len(ansatz))*np.pi/6
E_full = Energy_matrix(thetas, N, matrix_H, matrix_ansatz, K)
G_K = G_k(N, H, ansatz, K)

E_order = []
o_max=len(ansatz)*2
s = s_dict(N, ansatz, K, o_max)
print(s_dict)
for o in range(1,o_max+1):
    E_approx = energy(thetas, ansatz, s, G_K, o)
    E_order += [E_approx]
    print(f"{o}/{o_max}")
print(E_order)
plt.plot(range(1,o_max+1), np.log(np.abs(E_order-E_full)))
plt.title(f"N = {N}, len(ansatz) = {len(ansatz)}")
plt.ylabel("log(E_order-E_full)")
plt.xlabel("order (o)")